<div style="text-align: right;">
  <img src="https://raw.githubusercontent.com/exasol/ai-lab/refs/heads/main/assets/Exasol_Logo_2025_Dark.svg" style="width:100px; margin: 20px;" />
</div>

# Create and Cutomize the Script Languages Container
A [Script-Language Container](https://github.com/exasol/script-languages-release)(SLC) for the Exasol database consists of a Linux container with a complete Linux distribution and all required libraries, such as a script client. The script client is responsible for the communication with the  database and for executing the script code. It allows to also include user specific libraries which can then be used from within the UDFs.

## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the AI-Lab](../main_config.ipynb).

### Open Secure Configuration Storage



In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

### Check that we don't have a SaaS backend configured

In [ ]:
from exasol.nb_connector.ai_lab_config import StorageBackend, AILabConfig
if ai_lab_config.get(AILabConfig.storage_backend, ) == StorageBackend.saas.name:
    popup_message(f"This tutorial will not work correctly with a SaaS as backend. You can export the Script-Languages-Container to a local file, but the upload to the database will fail.")


## Specific Configuration for this Tutorial

For this tutorial we need the build definition of a Script-Languages container.

The tutorial will clone the Script-Languages-Release repository and use a specifc version.
First we import a convenience class from the notebook connector, called `ScriptLanguagesContainer`.

In [ ]:
from exasol.nb_connector.slc import ScriptLanguageContainer

This class provides a constant pointing to the Github release used in the current version.

In [ ]:
ScriptLanguageContainer.GITHUB_URL

The class `ScriptLanguageContainer` also provides a function to fetch and show all available template flavors:

In [ ]:
ScriptLanguageContainer.list_available_flavors()

### Define the flavor for our Script Languages Container

Now let's the define the flavor we want to use in this tutorial. In this tutorial we will use the [template-Exasol-all-python-3.10](https://github.com/exasol/script-languages/blob/master/flavors/template-Exasol-all-python-3.10/FLAVOR_DESCRIPTION.md) flavor.


In [ ]:
flavor = "template-Exasol-all-python-3.10"

### Create an instance of class ScriptLanguagesContainer

The following cell creates an instance of class `ScriptLanguageContainer` from the notebook-connector,
which enables using the`exaslct` in the AI Lab in a convenient way.
Creating this instance will:
* Check that no other entries exist in Secret Store
* Add all necessary entries to the Secret Store
* Clone the [script-languages-release](https://github.com/exasol/script-languages-release) repository

In [ ]:
slc = ScriptLanguageContainer.create(secrets=ai_lab_config, name="sample_slc_6", flavor=flavor)
print(f"{slc.name} is ready.")

Now we can continue with using the script-languages-container tool

# Export the flavor as is

Exasol [User Defined Functions](https://docs.exasol.com/db/latest/database_concepts/udf_scripts.htm) (UDFs) enable embedding user code into SQL statements. Each Python UDF runs in a so-called [Script-Languages-Container](https://docs.exasol.com/db/latest/database_concepts/udf_scripts/adding_new_packages_script_languages.htm) (SLC). Exasol provides default SLCs with some preinstalled PYthon packages but also allows users to create their own SLCs, e.g. by adding additional dependencies.

This tutorial shows how to build a Script-Languages-Container (SLC) from a base flavor (without any modification) and write the result to a tar gz file. The base flavor `template-Exasol-all-python-3.10` is part of the Script-Languages Release repository.

`exaslct` uses the flavor description to build a Docker image which is called the `release` image. `exaslct` can export the content of this Docker image then to a tar gz file.

In [ ]:
slc.flavor

### Export the flavor

Now execute the `export` step. The command builds the docker image and exports the Docker image to the export directory.

You could run the same on the command line with the `exaslct` tool:
```
cd <slc_dir>
./exaslct export --flavor-path flavors/<flavor_path>
```

In [ ]:
slc.export()

### Import Some utility functions


In [ ]:
%run ../script_languages_container/utils/file_system_ui.ipynb

#### Check the result
The following command will show the resulting files of the export command:

In [ ]:

show_directory_content(slc.workspace.export_path)

## Customize the SLC Flavour

We need the XGBoost module in our SLC which is not included in the current flavour that we just used to create our SLC. In order to do that we need to customize the flavour and export again.


### Flavor Customization Build Step

`exasclt` consists of multiple build steps. By a build step here we mean a file structure which serves as an input for a certain stage of the building process of the script-languages-container. See [Advanced Topics](../script_languages_container/advanced.ipynb) for more details.

Build step **flavor_customization** is defined by a Dockerfile and several package lists. We recommend to add new packages to the package lists and only modify the Dockerfile if you need very specific changes, like adding additional resources.

In [ ]:
show_directory_content(slc.flavor_path / "flavor_customization")

The Dockerfile consists of two parts. The first part installs the packages from the package lists and should only be changed with care. The second part is free for your changes. Read the description in the Dockerfile carefully to find out what you can and shouldn't do.

#### Package Lists
The package lists have a unified format. Each line consists of the package name and the package version separated by the pipe character `|`, e.g `xgboost|2.0.3`. You can comment out a whole line by adding a hash character `#` the beginning of the line. You can also add a trailing comment to a package definition by adding `#` after the package definition. We usually recommend to install a specific package version to avoid surprises about which version actually gets installed.

In [ ]:
show_files([slc.custom_pip_file])

We are now going to append Python package "xgboost" to one of the package lists by adding `xgboost|2.0.3` and `scikit-learn|1.5.0` to file `flavor_customization/packages/python3_pip_packages`. 
Notes:
 - running the following command multiple times will iteratively append the packages
 - you can also click on the link and modify the file directly

In [ ]:
from exasol.nb_connector.slc import PipPackageDefinition
xgboost_pkg = PipPackageDefinition(pkg="xgboost", version="2.0.3")
scikit_learn_pkg = PipPackageDefinition(pkg="scikit-learn", version="1.5.0")
slc.append_custom_pip_packages([xgboost_pkg, scikit_learn_pkg])
show_files([slc.custom_pip_file])

#### Rebuilding the customized Flavor

After changing the flavor you need to rebuild it. You can do it by running `export` again. Exaslct automatically recognizes that the flavor has changed and builds a new version of the container. Don't get confused by the warnings: `exaslct` first tries to find the cached docker images (see [Advanced Topics](./advanced.ipynb)), but as the content has changed, the cached image is not available, and the docker service returns a 404 error message.

In [ ]:
slc.export()

Lets check the resulting tar gz file:

In [ ]:
show_directory_content(slc.workspace.export_path)

#### Deploy the Container to the Database
To use the new container you need to upload it to the BucketFS. If the build machine has access to the BucketFS you do it with the `exaslct` upload command, as shown below. Otherwise you need to install the script-languages-container manually: 
1. Transfer the container tar gz file of the previous step to a machine that has access to the BucketFS. 
2. From that machine upload it via curl, as described in our [documentation](https://docs.exasol.com/db/latest/database_concepts/udf_scripts/adding_new_packages_script_languages.htm).

With the following command you upload the new script language container.
You could run the same on the command line with the `exaslct` tool:
```
cd <slc_dir>
./exaslct deploy --flavor-path flavors/<flavor_path> --bucketfs-host <bucketfs_host> --bucketfs-port <bucketfs_port> --bucketfs-user <bucketfs_username> --bucketfs-password <bucketfs_password> --bucketfs-name <bucketfs_name> --bucket <bucket_name>
```

In [ ]:
slc.deploy()

This command also stores the activation statement in the ai-lab-config. You can verify it with:


In [ ]:
slc.activation_key

The syntax of the activation statement is: `alias=url`. The activation key will be used in the `ALTER_SESSION` or `ALTER_SYSTEM` commands to "register" the script-language-container for usage in the UDFs.

You can generate the SQL commands for the activation with the following command line:
```
cd <slc_dir>
./exaslct generate-language-activation --flavor-path flavors/<flavor_path> --bucketfs-name <bucketfs_name> --bucket-name <bucket_name> --container-name <container_name> --path-in-bucket <path_in_bucket>
```

We can now use our SLC in our UDFs